# Control Motor

In [ ]:
from picomotor import MSerial

In [ ]:
m = MSerial('COM11', echo=True, wait=0.1)

In [ ]:
m.echo = False

In [ ]:
r = m.status()
r

In [ ]:
m.unit

In [ ]:
donor_unit = dict(x=787, y=588, detector='donor')
accept_unit = dict(x=625, y=641, detector='acceptor')

In [ ]:
m.unit = donor_unit

In [ ]:
m.unit

In [ ]:
m.unit = accept_unit

In [ ]:
m.unit

In [ ]:
vel, acc = 500, 10000
settings = [
    'acc a1 0=%d' % acc,
    'acc a1 1=%d' % acc,
    'vel a1 0=%d' % vel,
    'vel a1 1=%d' % vel,   
]
for s in settings:
    m.serial.flush()
    print(m.sendrecv(s))

In [ ]:
m.sendrecv('vel')
m.sendrecv('acc')

# GUI

In [ ]:
from IPython.display import display, Javascript
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, HBox, VBox, Box, Label
import ipywidgets as widgets
widgets.__version__

In [ ]:
items_base = dict(width='auto', height='40px', margin='6px')
items_layout = Layout(flex='1 1 auto', **items_base)

detector = widgets.ToggleButton(description="Donor", value=True, button_style='success',
                                layout=items_layout)
axis = widgets.ToggleButtons(
    options=['X', 'Y'],
    value='X',
    description='Axis:',
    disabled=False,
    layout=Layout(flex='1 0 auto', height='auto', margin='5px'),
)
delta_wid = widgets.BoundedFloatText(description='Delta:', layout=items_layout)
movebutton = widgets.Button(description="Move!", layout=items_layout)
out = widgets.Textarea(
    value='',
    description='Received:',
    layout=Layout(flex='1 1 auto'),
)

port = HBox([Label('Port:',
                   layout=Layout(flex='1 1 auto', width='6em', height='auto', margin='5px')), 
             widgets.Text(
                    value='COM11',
                    description='',
                    disabled=False,
                    layout=Layout(flex='1 1 auto', width='6em', height='auto', margin='5px'))]
           )
connect =  widgets.Button(description="Connect", layout=items_layout)
status =  widgets.Button(description="Check Status", layout=items_layout)
status.button_style = 'info'

vb_layout = Layout(flex='1 1 auto')
vbox = VBox([detector, axis, delta_wid, movebutton], layout=vb_layout)

vba_layout = Layout(flex='1 1 auto', justify_content='space-between')
vboxa = VBox([port, connect, status], layout=vba_layout)

ctrls = HBox([vboxa, vbox], layout=Layout(flex='1 0 auto'))
recv = widgets.Text(
    value='',
    description='Received:',
    layout=Layout(flex='1 1 auto', width='95%'),
)
sent = widgets.Text(
    value='',
    description='Sent:',
    layout=Layout(flex='1 1 auto', width='95%'),
)
sentrecv = VBox([sent, recv])
main = VBox([ctrls, recv])
tab = widgets.Tab(children=[vboxa, vbox])
tab.set_title(0, 'Configure')
tab.set_title(1, 'Move')
VBox([tab, sentrecv],
        layout=Layout(overflow_x='scroll', max_width='45em', 
                      flex='1 1 auto', ))#border='solid'))


In [ ]:
js_dialog_code = """require(
    ["base/js/dialog"], 
    function(dialog) {
        dialog.modal({
            title: 'Driver Connection',
            body: 'Please check that driver is connected to the %s detector!',
            buttons: {
                'OK': {}
            }
        });
    }
);"""

In [ ]:
def on_button_clicked(b):
    if b['new']:
        b['owner'].button_style = 'success'
        b['owner'].description = 'Donor'
        m.unit = donor_unit
    else:
        b['owner'].button_style = 'danger'
        b['owner'].description = 'Acceptor'
        m.unit = acceptor_unit
    display(Javascript(js_dialog_code % b['owner'].description))
detector.observe(on_button_clicked, names='value')

def move(b):
    m.move(delta_wid.value, axis=axis.value, vel=200, acc=1000)
movebutton.observe(on_axis_changed, names='value')